# Deutsch-Jozsa Algorithm

## Problem statement:

In the Deutsch–Jozsa problem, we are given a black box quantum computer known as an oracle that implements some function 

$$f(\{0,1\}^{n} = \{0,1\})$$

The function takes n-digit binary values as input and produces either a 0 or a 1 as output for each such value. We are promised that the function is either constant or balanced:
+ constant (0 on all outputs or 1 on all outputs) 
+ balanced (returns 1 for half of the input domain and 0 for the other half).

The task then is to determine if f is constant or balanced by using the oracle.


### Example:

For example, if $n = 3$, then the following functions are examples of constant function and balance function:

|$x$|   $$f(x)$$ constant|   $$f(x)$$ constant|   $$f(x)$$ balance|   $$f(x)$$ balance|   $$f(x)$$ balance|
|---|------|------|------|------|------|
|000|	0|	1|	1|	1|	1|
|001|	0|	1|	1|	1|	0|
|010|	0|	1|	1|	0|	1|
|011|	0|	1|	1|	0|	0|
|100|	0|	1|	0|	1|	1|
|101|	0|	1|	0|	1|	0|
|110|	0|	1|	0|	0|	1|
|111|	0|	1|	0|	0|	0|



In [1]:
from braket.devices import LocalSimulator
from braket.circuits import Circuit
import random
import matplotlib.pyplot as plt

In [2]:
# define the quantum device.
# use LocalSimulator so that it can be run locally with Braket SDK installed.
device = LocalSimulator()

In [3]:
# define a simple function to generate random 0 or 1, 0 stands for constant function , 1 stands for balance function

def get_function_type():
    random_value = random.random()
    if random_value < 0.5:
        return 0
    else:
        return 1

In [4]:
# define a simple function to generate random n, n is from 1 to 8

def get_bit_number():
    return random.randint(1,8)

In [5]:
# use get_bit_number to generate random bit number
# bit_number = get_bit_number()

# for testing, set the bit_number to 6
bit_number = 6
bit_number

6

In [6]:
def generate_full_bit_string(bit_number):
    zero_string = '0' * bit_number
    result_list = list()
    for i in range(pow(2, bit_number)):
        cur_string = (zero_string + bin(i)[2:])[-bit_number:]
        result_list.append(cur_string)
    return result_list

In [7]:
full_bit_strings = generate_full_bit_string(bit_number)

In [8]:
def generate_input_circuit(source_list):
    
    input_circuit_list = list()
    
    for input_index, digit_string in enumerate(source_list):
        cur_circuit = Circuit()
        for reg_index, digit_value in enumerate(digit_string):
            if (digit_value == '0'):
                cur_circuit.i(reg_index)
            elif (digit_value == '1'):
                cur_circuit.x(reg_index)
            else:
                raise Exception('incorrect input value: \'' + digit_value + '\' in: ' + digit_string )
        
        input_circuit_list.append(cur_circuit)
        
    return input_circuit_list

In [9]:
input_circuit_list = generate_input_circuit(full_bit_strings)

In [10]:
# generate black box circuit for constant function

def generate_constant_function(bit_number):
    circuit = Circuit()
    # insert i gate for each of the input qbit, as many circuit compliers require continued qbit
    for i in range(bit_number):
        circuit.i(i)
        
    # building output qbit for the function
    # functions generating ouput 0 or 1 constantly are constant functions
    if random.random() < 0.5:
        circuit.i(bit_number)
    else:
        circuit.x(bit_number)
        
    return circuit

In [11]:
# testing the generated constant function 
constant_circuit = generate_constant_function(bit_number)
print(constant_circuit)

T  : |0|
        
q0 : -I-
        
q1 : -I-
        
q2 : -I-
        
q3 : -I-
        
q4 : -I-
        
q5 : -I-
        
q6 : -X-

T  : |0|


In [12]:
# generate black box circuit for balance function

def generate_balance_function(bit_number):
    circuit = Circuit()
    # insert i gate for each of the input qbit, as many circuit compliers require continued qbit
    for i in range(bit_number):
        circuit.i(i)
        
    # randomly select one qbit from input as the flag.
    # use cnot gate to connect the flag qbit and output qbit
    # if the input value of the flag qbit is 0, the output is 0
    # if the input value of the flag qbit is 1, the output qbit will be fliped to 1.
    control_bit_index = random.randint(0, bit_number-1)
        
    circuit.cnot(control_bit_index, bit_number)
        
    return circuit

In [13]:
# testing the generated constant function 
balance_circuit = generate_balance_function(bit_number)
print(balance_circuit)

T  : |0|1|
          
q0 : -I---
          
q1 : -I---
          
q2 : -I---
          
q3 : -I-C-
        | 
q4 : -I-|-
        | 
q5 : -I-|-
        | 
q6 : ---X-

T  : |0|1|


In [14]:
def check_balance_function(bit_number, balance_circuit):
    shot_number = 1000
    threshold = 0.8
    
    full_bit_strings = generate_full_bit_string(bit_number)
    
    input_circuit_list = generate_input_circuit(full_bit_strings)
    
    result_statistics = dict()
    result_statistics['0'] = 0
    result_statistics['1'] = 0


    for input_string, input_circuit in zip(full_bit_strings, input_circuit_list):

        circuit = input_circuit + balance_circuit

        task = device.run(circuit, shots=shot_number)
        result = task.result()
        
        result_key_list = result.measurement_counts.keys()
        result_number = len(result_key_list)
        
        result_item = result.measurement_counts.most_common(1)[0]
        
        
        result_string = result_item[0]
        result_times = result_item[1]
        
        if (result_times/shot_number < threshold):
            raise Exception('Result for ' + input_string + ' is not stable.')
        else:
            
            result_statistics[result_string[-1:]] = result_statistics[result_string[-1:]] + 1
            
           
    expect_total_number = pow(2, bit_number)
    
    if result_statistics['0'] == result_statistics['1'] == expect_total_number/2:
        return True
    else:
        return False
    
            
        

In [15]:
check_balance_function(bit_number, balance_circuit)

True

In [16]:
def get_random_function(bit_number):
    random_value = random.random()
    if random_value < 0.5:
        return ('constant', generate_constant_function(bit_number))
    else:
        return ('balance', generate_balance_function(bit_number))

In [17]:
def generate_deutsch_circuit(bit_number, secret_function):
    
    input_circuit = Circuit()
    
    input_circuit.x(bit_number)
    
    for i in range(bit_number+1):
        input_circuit.h(i)
        
    combine_circuit = input_circuit + secret_function
    
    for i in range(bit_number):
        combine_circuit.h(i)
        
    return combine_circuit


In [18]:
def check_function_type(bit_number, secret_circuit):
    deutsch_circuit = generate_deutsch_circuit(bit_number, secret_circuit)
    shot_number = 1

    task = device.run(deutsch_circuit, shots=shot_number)
    result = task.result()

    result_value = int(list(result.measurement_counts.keys())[0][0:-1])

    if result_value > 0:
        return ('balance', result_value)
    elif result_value == 0:
        return ('constant', result_value)


    


In [19]:
check_function_type(bit_number, constant_circuit)

('constant', 0)

In [20]:
check_function_type(bit_number, balance_circuit)

('balance', 100)

In [21]:
iter_number = 50

for i in range(iter_number):
    bit_number = get_bit_number()
    function_type, secret_circuit = get_random_function(bit_number)
    
    detected_function_type, result_value = check_function_type(bit_number, secret_circuit)
    
    if (function_type == detected_function_type):
        print ('Bit number:' + str(bit_number))
        print ('Answer is right')
        print ('Function type generated: \t\t' + function_type)
        print ('Checked result is: \t\t\t' + detected_function_type)
        print ('-------------------------------------')
        

Bit number:2
Answer is right
Function type generated: 		balance
Checked result is: 			balance
-------------------------------------
Bit number:2
Answer is right
Function type generated: 		balance
Checked result is: 			balance
-------------------------------------
Bit number:8
Answer is right
Function type generated: 		balance
Checked result is: 			balance
-------------------------------------
Bit number:7
Answer is right
Function type generated: 		constant
Checked result is: 			constant
-------------------------------------
Bit number:2
Answer is right
Function type generated: 		constant
Checked result is: 			constant
-------------------------------------
Bit number:6
Answer is right
Function type generated: 		balance
Checked result is: 			balance
-------------------------------------
Bit number:2
Answer is right
Function type generated: 		constant
Checked result is: 			constant
-------------------------------------
Bit number:5
Answer is right
Function type generated: 		balance
Checke